**1. Initialization**

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
!pip install -q tensorflow==2.0.0-beta1

import tensorflow as tf
import matplotlib.pyplot as plt

**2. Load data from google drive using numpy**

In [4]:
import numpy as np

from google.colab import drive
drive.mount('/gdrive')

!ls /gdrive/My\ Drive/Colab\ Notebooks/data

xy = np.loadtxt('/gdrive/My Drive/Colab Notebooks/data/data-01-test-score.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, x_data, len(x_data))
print(y_data.shape, y_data)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
data-01-test-score.csv	 data-03-diabetes.csv
data-02-stock_daily.csv  data-04-zoo.csv
(25, 3) [[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]
 [ 53.  46.  55.]
 [ 69.  74.  77.]
 [ 47.  56.  60.]
 [ 87.  79.  90.]
 [ 79.  70.  88.]
 [ 69.  70.  73.]
 [ 70.  65.  74.]
 [ 93.  95.  91.]
 [ 79.  80.  73.]
 [ 70.  73.  78.]
 [ 93.  89.  96.]
 [ 78.  75.  68.]
 [ 81.  90.  93.]
 [ 88.  92.  86.]
 [ 78.  83.  77.]
 [ 82.  86.  90.]
 [ 86.  82.  89.]
 [ 78.  83.  85.]
 [ 76.  83.  71.]
 [ 96.  93.  95.]] 25
(25, 1) [[152.]
 [185.]
 [180.]
 [196.]
 [142.]
 [101.]
 [149.]
 [115.]
 [175.]
 [164.]
 [141.]
 [141.]
 [184.]
 [152.]
 [148.]
 [192.]
 [147.]
 [183.]
 [177.]
 [159.]
 [177.]
 [175.]
 [175.]
 [149.]
 [192.]]


**3. Expecting your score**


In [0]:
X = tf.Variable(x_data, shape=[None, 3])
Y = tf.Variable(y_data, shape=[None, 1])

W = tf.Variable(tf.random.uniform(shape=[3, 1], minval=0, maxval=1, dtype=tf.float32, seed=3))
b = tf.Variable(tf.random.uniform(shape=[1], minval=0, maxval=1, dtype=tf.float32, seed=3))

@tf.function
def hypothesis(x):
    return tf.matmul(x, W) + b

@tf.function
def cost(x, y):
    return tf.reduce_mean(tf.square(hypothesis(x) - y))

optimizer = tf.keras.optimizers.SGD(learning_rate=1e-5)
loss = lambda: cost(X, Y) 

for step in range(2001):
    optimizer.minimize(loss, var_list=[W, b])
    # if step % 10 == 0: 
    #     print("\tstep[{}] cost([W], b): {} Y: {}".format( \
    #         step, cost(X, Y).numpy(), hypothesis(X).numpy()))

print("\Your score will be : {}".format(hypothesis([[100., 70., 101.]]).numpy()))
print("\Other scores will be : {}".format(hypothesis([[60., 70., 110.], [90., 100., 80.]]).numpy()))

\Your score will be : [[185.94519]]
\Other scores will be : [[178.8464 ]
 [176.46725]]


**4. Loading data using tf's queue runners**

In [57]:
X = tf.Variable(x_data, shape=[None, 3])
Y = tf.Variable(y_data, shape=[None, 1])

W = tf.Variable(tf.random.uniform(shape=[3, 1], minval=0, maxval=1, dtype=tf.float32, seed=3))
b = tf.Variable(tf.random.uniform(shape=[1], minval=0, maxval=1, dtype=tf.float32, seed=3))

@tf.function
def hypothesis(x):
    return tf.matmul(x, W) + b

@tf.function
def cost(x, y):
    return tf.reduce_mean(tf.square(hypothesis(x) - y))

optimizer = tf.keras.optimizers.SGD(learning_rate=1e-5)

dataset = tf.data.Dataset.from_tensor_slices(xy)

# pre-processing
dataset = dataset.map(
    lambda x: list([x[0:-1], x[-1:]]))
# dataset = dataset.repeat()
# dataset = dataset.shuffle()
dataset = dataset.batch(10)

for step in range(2001):
    for element in dataset: 
        X = element[0]
        Y = element[1]

        loss = lambda: cost(X, Y) 
        optimizer.minimize(loss, var_list=[W, b])
    # if step % 10 == 0: 
    #     print("\tstep[{}] cost([W], b): {} Y: {}".format( \
    #         step, cost(X, Y).numpy(), hypothesis(X).numpy()))

print("\Your score will be : {}".format(hypothesis([[100., 70., 101.]]).numpy()))
print("\Other scores will be : {}".format(hypothesis([[60., 70., 110.], [90., 100., 80.]]).numpy()))

	step[0] cost([W], b): 3154.20458984375 Y: [[ 98.62864 ]
 [118.65933 ]
 [116.78015 ]
 [127.97078 ]
 [ 89.90197 ]
 [ 67.56374 ]
 [ 96.97117 ]
 [ 73.52328 ]
 [111.913445]
 [105.44816 ]]
	step[0] cost([W], b): 1240.7041015625 Y: [[112.63794 ]
 [111.42683 ]
 [146.12782 ]
 [120.364716]
 [118.362526]
 [147.362   ]
 [113.71069 ]
 [141.7177  ]
 [139.17139 ]
 [124.66887 ]]
	step[0] cost([W], b): 497.94097900390625 Y: [[154.9035 ]
 [153.39966]
 [147.45682]
 [134.5025 ]
 [168.20776]]
\Your score will be : [[162.71448]]
\Other scores will be : [[155.6316 ]
 [156.60701]]
